In [ ]:
import numpy as np
import sys
sys.path.insert(1, '/braintree/home/andrewci/brain-score/')
from candidate_models.model_commitments import brain_translated_pool
#sys.path.insert(1, '/braintree/home/andrewci/brain-score/brainscore/benchmarks')
from cirincione_stim_common import load_stim_info
from brainscore.benchmarks.screen import place_on_screen
from brainscore.benchmarks._properties_common import get_stimulus_position, \
    filter_receptive_fields, firing_rates_affine, record_from_model, get_firing_rates, get_stimulus_position, filter_receptive_fields
import matplotlib.pyplot as plt
from cirincione2022_zhou2000_TESTING import modified_get_firing_rates, modified_record_from_model, load_model_and_BO_stimulus, BO_optimization_and_standard_test

In [2]:
optim_test_dir = '/braintree/data2/active/users/andrewci/stimulus_sets/dicarlo.Cirincione2022_border_ownership_optimization_test'
standard_test_dir = '/braintree/data2/active/users/andrewci/stimulus_sets/dicarlo.Cirincione2022_border_ownership_standard_test'
BO_STIM_NAME = 'dicarlo.Cirincione2022_border_ownership_standard_test'
BO_OPTIM_NAME = 'dicarlo.Cirincione2022_border_ownership_optimization_test'
#BLANK_STIM_NAME = 'dicarlo.Marques2020_blank'

In [3]:
model, optimization_test_responses, standard_test_responses = load_model_and_BO_stimulus(BO_OPTIM_NAME=BO_OPTIM_NAME, optim_test_dir=optim_test_dir,
            BO_STIM_NAME=BO_STIM_NAME, standard_test_dir=standard_test_dir)

alexnet is accessed again and reloaded
